# Performance, and using JIT 

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt 
import numpy as np 
import alpro 
import time

In [2]:
imp = ["c", "numba", "numba"]

for implementation in imp:

    s = alpro.Survival("1275b", implementation = implementation)
    s.init_model()
    s.set_params(1e-12 * 1e-9, 1e-13)
    s.domain.create_box_array(1800, 0, s.coherence_func, r0=0)
    energies = np.logspace(3,4,1000)
    t1 = time.time()
    P, Pradial = s.propagate(s.domain, energies, pol="both")
    print (implementation, time.time() - t1)

c 0.08486390113830566
numba 3.0036709308624268
numba 0.11480879783630371
